In [10]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

In [11]:
# Load the datasets
train_df = pd.read_csv('/Users/alitahseen/Desktop/FYP-2024/Machine_learning/Datafiles/Final_Train.csv')
test_df = pd.read_csv('/Users/alitahseen/Desktop/FYP-2024/Machine_learning/Datafiles/Final_test.csv')
validation_df = pd.read_csv('/Users/alitahseen/Desktop/FYP-2024/Machine_learning/Datafiles/Final_Validation.csv')

In [12]:
# Function to preprocess datasets
def preprocess_data(df):
    df['local_time'] = pd.to_datetime(df['local_time'])
    for time_unit in ['Year', 'Month', 'Day', 'Hour']:
        df[time_unit] = getattr(df['local_time'].dt, time_unit.lower())
    return df.drop('local_time', axis=1)

In [13]:
# Preprocess all datasets
train_df = preprocess_data(train_df)
test_df = preprocess_data(test_df)
validation_df = preprocess_data(validation_df)

def add_lagged_features(df, n_lags=3):
    for lag in range(1, n_lags + 1):
        df[f'Average_Temp_lag_{lag}'] = df['Average_Temp'].shift(lag)
    return df.dropna().reset_index(drop=True)

train_df = add_lagged_features(train_df)
validation_df = add_lagged_features(validation_df)
test_df = add_lagged_features(test_df)


In [14]:
# Preparing the feature sets and target variables
def get_features_targets(df):
    feature_cols = ['Year', 'Month', 'Day', 'Hour', 'Average_Temp'] + [f'Average_Temp_lag_{lag}' for lag in range(1, 4)]  # Assuming 3 lags
    X = df[feature_cols]
    y = df['MW']
    return X, y


X_train, y_train = get_features_targets(train_df)
X_validation, y_validation = get_features_targets(validation_df)
X_test, y_test = get_features_targets(test_df)

In [15]:
# Evaluation Function

def calculate_metrics(actual, predicted, lower_bound=0, upper_bound=100, iqr_multiplier=1.5):
    # Excluding negative actual values if considered invalid
    valid_indices = actual > lower_bound
    actual = actual[valid_indices]
    predicted = predicted[valid_indices]

    # Calculate MAE and RMSE
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))

    # Thresholding for outlier exclusion based on IQR
    q1, q3 = np.percentile(actual, [25, 75])
    iqr = q3 - q1
    outlier_threshold_upper = q3 + (iqr * iqr_multiplier)
    outlier_threshold_lower = q1 - (iqr * iqr_multiplier)

    valid_indices_for_mape = (actual >= outlier_threshold_lower) & (actual <= outlier_threshold_upper)
    filtered_actual = actual[valid_indices_for_mape]
    filtered_predicted = predicted[valid_indices_for_mape]

    # Calculate Modified MAPE with capped at 100%
    if len(filtered_actual) > 0:
        percentage_errors = np.abs((filtered_predicted - filtered_actual) / filtered_actual) * 100
        percentage_errors = np.clip(percentage_errors, None, upper_bound) 
        mape = np.mean(percentage_errors)
    else:
        mape = np.nan

    # Calculate sMAPE
    smape = 100/len(actual) * np.sum(2 * np.abs(predicted - actual) / (np.abs(actual) + np.abs(predicted)))

    return mae, mape, smape, rmse


In [16]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import numpy as np
from joblib import dump, load

pipeline = make_pipeline(StandardScaler(), Ridge())

alpha_values = np.logspace(-3, 6, 100)

# Setting up the GridSearchCV object with 5-fold cross-validation
parameters = {'ridge__alpha': alpha_values}
grid_search = GridSearchCV(pipeline, parameters, cv=5, scoring='neg_mean_absolute_error', verbose=2)


grid_search.fit(X_train, y_train)

# Retrieving the best model from grid search
best_model_Ridge = grid_search.best_estimator_

# Save the model to a file
model_filename = '/Users/alitahseen/Desktop/FYP-2024/Machine_learning/notebooks/Trained_Models/best_ridge_model_V2.joblib'
dump(best_model_Ridge, model_filename)


Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END .................................ridge__alpha=0.001; total time=   0.0s
[CV] END .................................ridge__alpha=0.001; total time=   0.0s
[CV] END .................................ridge__alpha=0.001; total time=   0.0s
[CV] END .................................ridge__alpha=0.001; total time=   0.0s
[CV] END .................................ridge__alpha=0.001; total time=   0.0s
[CV] END .................ridge__alpha=0.0012328467394420659; total time=   0.0s
[CV] END .................ridge__alpha=0.0012328467394420659; total time=   0.0s
[CV] END .................ridge__alpha=0.0012328467394420659; total time=   0.0s
[CV] END .................ridge__alpha=0.0012328467394420659; total time=   0.0s
[CV] END .................ridge__alpha=0.0012328467394420659; total time=   0.0s
[CV] END .................ridge__alpha=0.0015199110829529332; total time=   0.0s
[CV] END .................ridge__alpha=0.00151

['/Users/alitahseen/Desktop/FYP-2024/Machine_learning/notebooks/Trained_Models/best_ridge_model_V2.joblib']

In [17]:
# Predict on the validation set using the best model
model_filename = '/Users/alitahseen/Desktop/FYP-2024/Machine_learning/notebooks/Trained_Models/best_ridge_model_V2.joblib'
best_model_Ridge_loaded = load(model_filename)
predictions_validation = best_model_Ridge_loaded.predict(X_validation)
metrics_validation = calculate_metrics(y_validation, predictions_validation)
print("Validation Metrics after tuning (MAE, MAPE, sMAPE, RMSE):", metrics_validation)


Validation Metrics after tuning (MAE, MAPE, sMAPE, RMSE): (60.60834317236927, 79.8685954531016, 75.46616192467977, 68.257139703947)


In [18]:
# Evaluation on test set
predictions_test = best_model_Ridge_loaded.predict(X_test)
metrics_test = calculate_metrics(y_test, predictions_test)

print("Test Metrics with Ridge Regression (MAE, MAPE, sMAPE, RMSE):", metrics_test)


Test Metrics with Ridge Regression (MAE, MAPE, sMAPE, RMSE): (60.727444004173286, 88.86298561557933, 74.64496301891322, 64.0667099543779)
